In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
import logging
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

In [ ]:
%%time
preproced_short_X = np.load('/home/schirrmr/code/auto-diagnosis/preproced-short-X.npy')

In [ ]:

import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.util import np_to_var
def clean_jumps(x, window_len, threshold, expected, cuda):
    x_var = np_to_var([x])
    if cuda:
        x_var = x_var.cuda()

    maxs = F.max_pool1d(x_var,window_len, stride=1)
    mins = F.max_pool1d(-x_var,window_len, stride=1)

    diffs = maxs + mins
    large_diffs = (diffs > threshold).type_as(diffs) * diffs
    padded = F.pad(large_diffs.unsqueeze(0), (window_len-1,window_len-1, 0,0), 'constant', 0)
    max_diffs = th.max(padded[:,:,:,window_len-1:], padded[:,:,:,:-window_len+1]).unsqueeze(0)
    max_diffs = th.clamp(max_diffs, min=expected)
    x_var = x_var * (expected / max_diffs)

    x = x_var.data.cpu().numpy()[0]
    return x

In [ ]:
x = preproced_short_X[276]

In [ ]:
plt.plot(x[:,17000:18000].T);

In [ ]:
window_len = 25
threshold = 10
expected = 1
cuda = True
cleaned_x = clean_jumps(x,window_len, threshold,expected, cuda)

In [ ]:
plt.plot(cleaned_x[:,17000:18000].T);

In [ ]:
x = preproced_short_X[1739]
plt.plot(x[:,13000:14000].T);
plt.figure()
cleaned_x = clean_jumps(x,window_len, threshold,expected, cuda)
plt.plot(cleaned_x[:,13000:14000].T);

In [ ]:
def running_mean(arr, window_len, axis=0):
    # adapted from http://stackoverflow.com/a/27681394/1469195
    # need to pad to get correct first value also
    arr_padded = np.insert(arr,0,values=0,axis=axis)
    cumsum = np.cumsum(arr_padded,axis=axis)
    later_sums = np.take(cumsum, range(window_len, arr_padded.shape[axis]), 
        axis=axis)
    earlier_sums = np.take(cumsum, range(0, arr_padded.shape[axis] - window_len), 
        axis=axis)

    moving_average = (later_sums - earlier_sums) / float(window_len)
    return moving_average

def padded_moving_mean(arr, axis, n_window):
    """Pads by replicating n_window first and last elements
    and putting them at end and start (no reflection)"""
    start_pad_inds = list(range(0, n_window // 2))
    end_pad_inds = list(range(arr.shape[axis] - (n_window // 2),
                              arr.shape[axis]))
    arr = np.concatenate((arr.take(start_pad_inds, axis=axis),
                          arr,
                          arr.take(end_pad_inds, axis=axis)),
                         axis=axis)
    mov_mean = running_mean(arr, window_len=n_window, axis=axis)
    return mov_mean

def padded_moving_demean(arr, axis, n_window):
    assert arr.dtype != np.float16
    assert n_window % 2 == 1
    mov_mean = padded_moving_mean(arr, axis, n_window=n_window)
    arr = arr - mov_mean
    return arr

def padded_moving_divide_square(arr, axis, n_window, eps=0.1):
    """Pads by replicating n_window first and last elements
    and putting them at end and start (no reflection)"""
    assert arr.dtype != np.float16
    assert n_window % 2 == 1
    mov_mean_sqr = padded_moving_mean(np.square(arr), axis=axis, n_window=n_window)
    arr = arr / np.maximum(np.sqrt(mov_mean_sqr), eps)
    return arr

def padded_moving_standardize(arr, axis, n_window, eps=0.1):
    assert arr.dtype != np.float16
    assert n_window % 2 == 1
    arr = padded_moving_demean(arr, axis=axis, n_window=n_window)
    arr = padded_moving_divide_square(arr, axis=axis, n_window=n_window, eps=eps)
    return arr


In [ ]:
%%prun
padded_moving_standardize(x,axis=1, n_window=201)


In [ ]:
from braindecode.models.deep4 import Deep4Net
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.models.util import to_dense_prediction_model
input_time_length = 2000
in_chans = 21
n_classes = 2
model = Deep4Net(in_chans, n_classes, input_time_length=input_time_length,
    final_conv_length=2).create_network()
#model = ShallowFBCSPNet(in_chans, n_classes, input_time_length=input_time_length,
#                            final_conv_length=40).create_network()
to_dense_prediction_model(model)
log.info("Model:\n{:s}".format(str(model)))
if cuda:
    model.cuda()

from braindecode.torch_ext.util import np_to_var
# determine output size
test_input = np_to_var(np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
log.info("{:d} predictions per input/trial".format(n_preds_per_input))
log.info("{:d} receptive field size".format(input_time_length - n_preds_per_input + 1))

In [ ]:
from braindecode.datautil.signalproc import lowpass_cnt, highpass_cnt

In [ ]:
x_bandpassed = highpass_cnt(x, 0.1, 100,5)

plt.plot(x_bandpassed[:,13000:14000].T);

In [ ]:
x_bandpassed = lowpass_cnt(x, 40, 100,15)
plt.plot(x_bandpassed[:,13000:14000].T);

In [ ]:
x_bandpassed = bandpass_cnt(x, 0.1, 40, 100,5)
plt.plot(x_bandpassed[:,13000:14000].T);